## Captcha Detection

In this Image processing project, we have a [dataset](https://drive.google.com/drive/folders/16sQxbNpJiyGsdv1QGDnCA6SxWgQUj8tz?usp=drive_link) which contain train and test datasets.


### Goals:
- Train the model to detect rotation angle
- test the trained model on test dataset
- calculate the result using ```F1_score``` method

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import cv2
import os

from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
image_height, image_width = 128, 128
num_channels = 3

def load_data(image_dir, labels_file):
    images = []
    labels = []

    with open(labels_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            image_name, angle = line.strip().split(",")
            image_path = os.path.join(image_dir, image_name)

            image = cv2.imread(image_path)
            image = cv2.resize(image, (image_height, image_width))
            image = img_to_array(image)
            images.append(image)
            labels.append(float(angle))

    return np.array(images), np.array(labels)

image_dir = '/content/drive/MyDrive/captcha_init/train_images'
labels_file = '/content/drive/MyDrive/captcha_init/train_info.txt'
images, labels = load_data(image_dir, labels_file)

images = images / 255.0


train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)



In [ ]:
def create_model():
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))

    model.add(layers.Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

model = create_model()

history = model.fit(train_images, train_labels, epochs=1, validation_data=(val_images, val_labels))

model.save('angle_prediction_model.h5')

def predict_angle(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_height, image_width))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    predicted_angle = model.predict(image)[0][0]

    return predicted_angle





/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 90s 833ms/step - loss: 14436.2715 - val_loss: 10703.6064


In [ ]:
from PIL import Image

folder = '/content/drive/MyDrive/captcha_init/test_images'

result = []
for name in os.listdir(folder):
  img = Image.open(os.path.join(folder, name))
  predicted_angle = predict_angle(os.path.join(folder, name))
  result.append({name: predicted_angle})

print(result)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━

In [ ]:
import pandas as pd

df = pd.DataFrame(result)

submission = df